# Semantic Kernel mit OpenBnB MCP Server-Integration

Dieses Notebook zeigt, wie man Semantic Kernel mit dem tatsächlichen OpenBnB MCP-Server verwendet, um mithilfe des MCPStdioPlugin nach echten Airbnb-Unterkünften zu suchen. Für den Zugriff auf LLM wird Azure AI Foundry genutzt. Um Ihre Umgebungsvariablen einzurichten, können Sie der [Setup-Lektion](/00-course-setup/README.md) folgen.


## Importieren Sie die benötigten Pakete


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Erstellen der MCP-Plugin-Verbindung

Wir werden eine Verbindung zum [OpenBnB MCP-Server](https://github.com/openbnb-org/mcp-server-airbnb) mithilfe von MCPStdioPlugin herstellen. Dieser Server bietet Suchfunktionen für Airbnb über das @openbnb/mcp-server-airbnb-Paket.


## Erstellen des Clients

In diesem Beispiel verwenden wir Azure AI Foundry für den Zugriff auf unser LLM. Stellen Sie sicher, dass Ihre Umgebungsvariablen korrekt eingerichtet sind.


## Umgebungskonfiguration

Konfigurieren Sie die Azure OpenAI-Einstellungen. Stellen Sie sicher, dass die folgenden Umgebungsvariablen gesetzt sind:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Verständnis der OpenBnB MCP-Integration

Dieses Notebook verbindet sich mit dem **realen OpenBnB MCP-Server**, der die tatsächliche Airbnb-Suchfunktionalität bereitstellt.

### So funktioniert es:

1. **MCPStdioPlugin**: Nutzt die Kommunikation über Standard-Ein-/Ausgabe mit dem MCP-Server
2. **Echtes NPM-Paket**: Lädt und führt `@openbnb/mcp-server-airbnb` über npx aus
3. **Echtzeitdaten**: Gibt tatsächliche Airbnb-Immobiliendaten aus deren APIs zurück
4. **Funktionsentdeckung**: Der Agent erkennt automatisch die verfügbaren Funktionen des MCP-Servers

### Verfügbare Funktionen:

Der OpenBnB MCP-Server stellt typischerweise folgende Funktionen bereit:
- **search_listings** - Suche nach Airbnb-Immobilien anhand von Standort und Kriterien
- **get_listing_details** - Abrufen detaillierter Informationen zu bestimmten Immobilien
- **check_availability** - Verfügbarkeit für bestimmte Daten prüfen
- **get_reviews** - Bewertungen für Immobilien abrufen
- **get_host_info** - Informationen über Gastgeber von Immobilien erhalten

### Voraussetzungen:

- **Node.js** auf Ihrem System installiert
- **Internetverbindung**, um das MCP-Server-Paket herunterzuladen
- **NPX** verfügbar (ist in Node.js enthalten)

### Verbindung testen:

Sie können den MCP-Server manuell testen, indem Sie folgendes ausführen:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Dies lädt den OpenBnB MCP-Server herunter und startet ihn, sodass Semantic Kernel sich verbinden und echte Airbnb-Daten abrufen kann.


## Ausführen des Agenten mit dem OpenBnB MCP-Server

Jetzt werden wir den KI-Agenten ausführen, der sich mit dem OpenBnB MCP-Server verbindet, um echte Airbnb-Unterkünfte in Stockholm für 2 Erwachsene und 1 Kind zu suchen. Sie können die Liste `user_inputs` gerne ändern, um die Suchkriterien anzupassen.


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

Zusammenfassung  
Herzlichen Glückwunsch! Sie haben erfolgreich einen KI-Agenten entwickelt, der mit der realen Unterkunftssuche über das Model Context Protocol (MCP) integriert ist:

Verwendete Technologien:  
- Semantic Kernel – Zum Aufbau intelligenter Agenten mit Azure OpenAI  
- Azure AI Foundry – Für LLM-Funktionen und Chat-Komplettierung  
- MCP (Model Context Protocol) – Für standardisierte Tool-Integration  
- OpenBnB MCP Server – Für echte Airbnb-Suchfunktionen  
- Node.js/NPX – Zum Ausführen des externen MCP-Servers  

Was Sie gelernt haben:  
- MCP-Integration: Verbindung von Semantic Kernel-Agenten mit externen MCP-Servern  
- Echtzeit-Datenzugriff: Suche nach tatsächlichen Airbnb-Unterkünften über Live-APIs  
- Protokollkommunikation: Verwendung von Stdio-Kommunikation zwischen Agent und MCP-Server  
- Funktionsentdeckung: Automatische Erkennung verfügbarer Funktionen von MCP-Servern  
- Streaming-Antworten: Erfassen und Protokollieren von Funktionsaufrufen in Echtzeit  
- HTML-Darstellung: Formatierung von Agentenantworten mit gestalteten Tabellen und interaktiven Anzeigen  

Nächste Schritte:  
- Integration zusätzlicher MCP-Server (Wetter, Flüge, Restaurants)  
- Aufbau eines Multi-Agenten-Systems, das MCP- und A2A-Protokolle kombiniert  
- Erstellung benutzerdefinierter MCP-Server für eigene Datenquellen  
- Implementierung von persistentem Gesprächsspeicher über Sitzungen hinweg  
- Bereitstellung des Agenten in Azure Functions mit MCP-Server-Orchestrierung  
- Hinzufügen von Benutzerauthentifizierung und Buchungsfunktionen  

Wichtige Vorteile der MCP-Architektur:  
- Standardisierung: Universelles Protokoll zur Verbindung von KI-Agenten mit externen Tools  
- Echtzeit-Daten: Zugriff auf aktuelle, live verfügbare Informationen aus verschiedenen Diensten  
- Erweiterbarkeit: Einfache Integration neuer Datenquellen und Tools  
- Interoperabilität: Funktioniert über verschiedene KI-Frameworks und Agentenplattformen hinweg  
- Trennung von Verantwortlichkeiten: Klare Unterscheidung zwischen KI-Logik und externem Datenzugriff  



---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
